In [19]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [20]:
mods = []

driver = webdriver.Chrome()

for page in range(1,251):
    url = f'https://www.curseforge.com/minecraft/search?page={page}&pageSize=20&sortBy=popularity&class=mc-mods'

    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    box = soup.find('div',class_='results-container')
    cards = box.find_all('div', class_='project-card')
    
    for i,element in enumerate(cards):
        mod = {}
        cates = []
        loadlst = []
        
        mod['name'] = element.find('span',class_='ellipsis').text
        mod['author'] = element.find('a',class_='author-name').text
        mod['latest_release'] = element.find('li',class_='detail-updated').text
        mod['created'] = element.find('li',class_='detail-created').text
        mod['size'] = element.find('li',class_='detail-size').text
        mod['latest_game_version'] = element.find('li',class_='detail-game-version').text
        cats = element.find('ul', class_='categories')
        for cat in cats.find_all('li'):
            cates.append(cat.text.strip())
        mod['categories'] = ' | '.join(cates[1:])
        
        overlay_href = element.find('a',class_='overlay-link')['href']
        
        overlay_link = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f"//a[@href='{overlay_href}']"))
        )
        
        driver.execute_script("arguments[0].scrollIntoView(true);", overlay_link)
        time.sleep(1)  # Small delay to ensure the element is in view
            
        # Try clicking the element using JavaScript to avoid interception
        driver.execute_script("arguments[0].click();", overlay_link)
        time.sleep(4)
            
        mod['downloads'] = driver.find_element(By.CSS_SELECTOR, 'li.detail-downloads').text
        time.sleep(2)

        try:
            loaders_box = driver.find_element(By.CSS_SELECTOR,'section#game-version-types-summary')
            time.sleep(2)
            driver.execute_script("arguments[0].scrollIntoView(true);", loaders_box)
            time.sleep(2)
            loaders = loaders_box.find_element(By.CSS_SELECTOR,'ul#project-game-version-types')
            time.sleep(2)
            loader = loaders.find_elements(By.CSS_SELECTOR,'li#game-version-type-item')
            time.sleep(2)

            combined_text = ', '.join([item.text for item in loader])

            mod['mod_loaders']= combined_text

        except Exception as e:
            mod['mod_loaders']= '-'

        time.sleep(2)
        
        driver.back()
        
        mods.append(mod)
        print(f"item: {i+1}")

    print(f"Page:{page}")

ds = pd.DataFrame(data=mods)

driver.quit()

In [ ]:
file_path = 'MCMods.csv'

ds.to_csv(file_path, index=False)